In [1]:
import openmm.app as app
import openmm as om
from openmm import unit


In [2]:

# Load PDB file and force field
pdf_file="/Users/kap037/Desktop/CSIRO-Malaysia-alphafold/scripts/hiv_protease_darunavir_receptor_ph_5.5.pdb"
pdb = app.PDBFile(pdf_file)
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')



In [ ]:


# Create System and Integrator (though not strictly needed for minimization, often present)
#https://github.com/openmm/openmm/issues/3782
#https://docs.openmm.org/latest/userguide/application/02_running_sims.html

#The assignment below combines the force field with the molecular topology loaded from the PDB file to create a complete mathematical description of the system we want to simulate.
#PME - Particle mesh ewald for long range electrostatic interaction calculation 
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME,
                                 nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)
integrator = om.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)

# Create Context
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# Perform energy minimization
print('Performing energy minimization...')
simulation.minimizeEnergy(maxIterations=500, tolerance=10.0*unit.kilojoule_per_mole/unit.nanometer)
print('Minimization complete.')



Performing energy minimization...
Minimization complete.


In [4]:


# Get minimized positions and save
state = simulation.context.getState(getPositions=True)
app.PDBFile.writeFile(simulation.topology, state.getPositions(), open('/Users/kap037/Desktop/CSIRO-Malaysia-alphafold/scripts/hiv_protease_darunavir_receptor_ph_5.5_minimized.pdb', 'w'))